In [1]:
!pip install --q datasets
!pip install --q --upgrade trl
!pip install --upgrade accelerate
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 9.0 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1


In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('NlpHUST/gpt2-vietnamese')
model = GPT2LMHeadModel.from_pretrained('NlpHUST/gpt2-vietnamese')

text = "Việt Nam là quốc gia có"
input_ids = tokenizer.encode(text, return_tensors='pt')
max_length = 100

sample_outputs = model.generate(input_ids,pad_token_id=tokenizer.eos_token_id,
                                   do_sample=True,
                                   max_length=max_length,
                                   min_length=max_length,
                                   top_k=40,
                                   num_beams=5,
                                   early_stopping=True,
                                   no_repeat_ngram_size=2,
                                   num_return_sequences=3)

for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))
    print('\n---')


tokenizer_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/854k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/512k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

>> Generated text 1

Việt Nam là quốc gia có nền kinh tế hàng đầu trên thế giới và được đánh giá là một trong những thị trường xuất khẩu lớn nhất của Việt Nam.
Theo số liệu thống kê của Tổng cục Hải quan, trong 6 tháng đầu năm 2018, tổng kim ngạch xuất nhập khẩu hàng hóa của cả nước ước đạt 2,12 tỷ USD, tăng 11,4% so với cùng kỳ năm 2017. Trong đó, xuất siêu đạt 1,6 tỷ đô la Mỹ, chiếm 13,2

---
>> Generated text 2

Việt Nam là quốc gia có nền kinh tế hàng đầu trên thế giới và được đánh giá là một trong những thị trường xuất khẩu lớn nhất của Việt Nam.
Theo số liệu thống kê của Tổng cục Hải quan, trong 6 tháng đầu năm 2018, tổng kim ngạch xuất nhập khẩu hàng hóa của cả nước ước đạt 2,12 tỷ USD, tăng 11,4% so với cùng kỳ năm 2017. Trong đó, xuất siêu đạt 1,6 tỷ đô la Mỹ, chiếm 5,8

---
>> Generated text 3

Việt Nam là quốc gia có nền kinh tế hàng đầu trên thế giới và được đánh giá là một trong những thị trường xuất khẩu lớn nhất của Việt Nam.
Theo số liệu thống kê của Tổng cục Hải quan, 

In [3]:
# Đếm số lượng tham số của model
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params_count = sum([torch.numel(p) for p in model_parameters])

print(f"Total trainable parameters: {params_count}")

Total trainable parameters: 124439808


In [4]:
import random

from datasets import load_dataset
open_instruct_dataset = load_dataset("5CD-AI/Vietnamese-Ecommerce-Alpaca", split="train")

max_seq_len = 100
total_data_points = len(open_instruct_dataset)
sample_size = 10000
random_indices = random.sample(range(total_data_points), sample_size)



# filter dataset to rows where the entire context length is less than or equal to max_seq_len,
# which is the size of the DeciLM-6B context window

open_instruct_dataset = open_instruct_dataset.select(random_indices)

dataset = open_instruct_dataset.filter(
    lambda example: (len(tokenizer(example.get("instruction", ""), truncation=True)["input_ids"]) +
                     len(tokenizer(example.get("input", ""), truncation=True)["input_ids"]) +
                     len(tokenizer(example.get("output", ""), truncation=True)["input_ids"])) <= max_seq_len)

ecommerce_alpaca.json:   0%|          | 0.00/84.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/69303 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
dataset

Dataset({
    features: ['output', 'instruction', 'input'],
    num_rows: 2377
})

In [6]:
def format_row_as_instruction_prompt(example):
    has_input = example.get('input', None) is not None

    if has_input:
        primer_prompt = ("Dưới đây là một hướng dẫn mô tả nhiệm vụ, kèm theo một đầu vào "
                         "cung cấp thêm ngữ cảnh. Viết một phản hồi hoàn chỉnh để thực hiện yêu cầu.")
        input_template = f"### Đầu vào: \n{example['input']}\n\n"
    else:
        primer_prompt = ("Dưới đây là một hướng dẫn mô tả nhiệm vụ. "
                         "Viết một phản hồi hoàn chỉnh để thực hiện yêu cầu.")
        input_template = ""

    instruction_template = f"### Hướng dẫn: \n{example['instruction']}\n\n"

    if example.get('output', None):
        response_template = f"### Phản hồi: \n{example['output']}\n\n"
    else:
        response_template = ""

    # Wrap the resulting string in a list
    return [f"{primer_prompt}\n\n{instruction_template}{input_template}{response_template}"]


In [7]:
print(format_row_as_instruction_prompt(dataset[0])[0])

Dưới đây là một hướng dẫn mô tả nhiệm vụ, kèm theo một đầu vào cung cấp thêm ngữ cảnh. Viết một phản hồi hoàn chỉnh để thực hiện yêu cầu.

### Hướng dẫn: 
Kết hợp hai mảng sau không có trùng lặp ["Dây giày thể thao", "Miếng lót giày", "Bàn chải đánh giày"] và ["Miếng lót giày", "Bàn chải đánh giày", "Móc khóa giày"]

### Đầu vào: 


### Phản hồi: 
Sau khi kết hợp hai mảng mà không có sự trùng lặp, mảng thu được là: ["Dây giày thể thao", "Miếng lót giày", "Bàn chải đánh giày", "Móc khóa giày"]




In [8]:
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

response_template = "### Phản hồi:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


In [9]:
import torch
from sklearn.model_selection import train_test_split

In [10]:
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.1, random_state=42)
train_dataset = dataset.select(train_indices)
val_dataset = dataset.select(val_indices)


from datasets import Dataset
from sklearn.model_selection import train_test_split

# Step 1: Format the Dataset
def format_dataset(dataset):
    """
    Format each row in the dataset using the format_row_as_instruction_prompt function
    and return a Hugging Face Dataset object.
    """
    # Create a dictionary to store formatted data
    formatted_data = {"text": []}

    for example in dataset:
        formatted = format_row_as_instruction_prompt(example)[0]
        formatted_data["text"].append(formatted)

    # Convert the dictionary into a Hugging Face Dataset
    return Dataset.from_dict(formatted_data)


# Step 2: Split the Dataset
def split_dataset(dataset, test_size=0.1, random_seed=42):
    """
    Split the dataset into training and validation sets.
    """
    train_indices, val_indices = train_test_split(range(len(dataset)), test_size=test_size, random_state=random_seed)
    train_dataset = dataset.select(train_indices)
    val_dataset = dataset.select(val_indices)
    return train_dataset, val_dataset

# Format the dataset
formatted_dataset = format_dataset(dataset)

# Split into train and validation datasets
train_dataset, val_dataset = split_dataset(formatted_dataset)

# Print a sample for verification
print(f"Sample formatted training example:\n{train_dataset[0]}")
print(f"Sample formatted validation example:\n{val_dataset[0]}")

Sample formatted training example:
{'text': 'Dưới đây là một hướng dẫn mô tả nhiệm vụ, kèm theo một đầu vào cung cấp thêm ngữ cảnh. Viết một phản hồi hoàn chỉnh để thực hiện yêu cầu.\n\n### Hướng dẫn: \nLập danh sách các thành phần chính của Coca-Cola\n\n### Đầu vào: \n\n\n### Phản hồi: \n1. Nước 2. Đường 3. Axit photphoric 4. Caffeine 5. Hương liệu 6. Màu thực phẩm 7. Chất bảo quản\n\n'}
Sample formatted validation example:
{'text': 'Dưới đây là một hướng dẫn mô tả nhiệm vụ, kèm theo một đầu vào cung cấp thêm ngữ cảnh. Viết một phản hồi hoàn chỉnh để thực hiện yêu cầu.\n\n### Hướng dẫn: \nViết một câu nhấn mạnh tầm quan trọng của việc đọc sách văn học.\n\n### Đầu vào: \n\n\n### Phản hồi: \nĐọc sách văn học là chìa khóa để mở rộng kiến thức, phát triển trí tuệ và nuôi dưỡng tâm hồn.\n\n'}


In [11]:
%%capture
!pip install evaluate

In [12]:
import evaluate

# Load a metric
metric = evaluate.load("accuracy") 

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits to predicted token indices
    predictions = logits.argmax(axis=-1)

    # Mask out ignored labels (e.g., -100)
    valid_indices = labels != -100

    # Flatten predictions and labels while filtering valid indices
    predictions = predictions[valid_indices]
    labels = labels[valid_indices]

    print("Predictions:", predictions)
    print("Labels:", labels)

    # Compute the metric
    return metric.compute(predictions=predictions, references=labels)

In [13]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [16]:
# thử instruction finetuning
tokenizer.pad_token = tokenizer.eos_token

model.resize_token_embeddings(len(tokenizer))

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    args=SFTConfig(
        output_dir="/tmp",
        num_train_epochs=1,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=10,
        evaluation_strategy="steps",
        eval_steps=50,
        fp16=True, 
        eval_accumulation_steps=4, 
    ),
    formatting_func=None,
    data_collator=collator,
)

import torch
torch.cuda.empty_cache()
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/2139 [00:00<?, ? examples/s]

Map:   0%|          | 0/238 [00:00<?, ? examples/s]

A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


Step,Training Loss,Validation Loss,Accuracy
50,No log,1.855463,0.042103
100,No log,1.800313,0.040584
150,No log,1.779424,0.040837
200,No log,1.771594,0.040837


Predictions: [ 203 1978 1080 ...  203  203  203]
Labels: [ 225  203 7843 ...   18  203  203]
Predictions: [ 203 7843 1080 ...  203  203  203]
Labels: [ 225  203 7843 ...   18  203  203]
Predictions: [ 203 1978 1080 ...  203  203  203]
Labels: [ 225  203 7843 ...   18  203  203]
Predictions: [ 203 1978 1080 ...  203  203  203]
Labels: [ 225  203 7843 ...   18  203  203]


TrainOutput(global_step=213, training_loss=1.750956503997946, metrics={'train_runtime': 527.7113, 'train_samples_per_second': 4.053, 'train_steps_per_second': 0.404, 'total_flos': 140926224384000.0, 'train_loss': 1.750956503997946, 'epoch': 0.9957924263674615})